In [2]:
import numpy as np
from numpy import linalg
import numpy.ma as ma
from sklearn.preprocessing import normalize


**Experiment Setup**

(a) Generate A as a random matrix with independent and identically distributed entries
drawn from the standard normal distribution. Normalize the columns of A.

In [ ]:
def generate_A(M,N):
    A = np.random.standard_normal(size = (M,N))
    A = normalize(A, axis=0, norm='l2') 
    return A

# M, N = 3, 5 # M < N
# generate_A(M,N)



array([[-0.74686253,  0.72678214, -0.50736219, -0.54021723, -0.06962175],
       [ 0.35724955,  0.12495016,  0.30043316, -0.29024427, -0.89612053],
       [-0.56086462, -0.67540742, -0.80766548, -0.78988835,  0.43831587]])

(b) Generate the sparse vector x with random support of cardinality s (i.e. s indices are
generated uniformly at random from integers 1 to N), and non-zero entries drawn as
uniform random variables in the range [−10,−1] ∪[1,10].

In [ ]:
def generate_x(s,N): #s < N, random # for now
    x = np.zeros(shape=(N,1))
    nonzeros = np.random.choice(N,size = s,replace=False) #replace = False: unique indices
    for i in nonzeros:
        x[i] = np.random.choice([-1,1]) * np.random.uniform(1,10)
    return x
generate_x(s=2,N=4)



array([[8.99492167],
       [0.        ],
       [0.        ],
       [6.20431957]])

(c) The entries of noise n are drawn independently from the normal distribution with stan-
dard deviation σ and zero mean.

In [ ]:
def generate_n(sigma,M,noise): 
    if noise == True:
        n = np.random.normal(loc=0,scale=sigma,size=(M,1))
    else:
        n = np.zeros(M,1)
    return n
# generate_n(sigma=1,M=M)

array([[ 0.90751962],
       [-0.58582287],
       [ 0.93755884]])

In [ ]:
def generate_y(M,N,s,sigma):
    A = generate_A(M,N)
    x = generate_x(s,N)
    n = generate_n(sigma,M)
    y = A.dot(x)+n
    return y, A, x, s
    
    

(d) For each cardinality s∈[1,smax], the average Normalized Error should be computed by
repeating step (a) to step (c) 2000 times and averaging the results over these 2000 Monte
Carlo runs.

In [ ]:
def OMP(A,y,threshold):
    r = y
    indices = []
    x_hat = np.zeros(shape=(A.shape[1],1))
    while linalg.norm(r,ord=2) > threshold:
        A_i = np.argmax(abs(A.T.dot(r)))
        indices.append(A_i)
        x_hat = np.linalg.inv(A.T.dot(A)).dot(A.T).dot(r)
        r = y - A.dot(x_hat)
    return r, A, x_hat


In [ ]:
def simulation(s=1,noise):
    normalized_error = 0
    y, A, x, s = generate_y()
    _, _, x_hat = OMP(y,A)
    for runs in range(2000):
        numerator = np.linalg.norm(x-x_hat,ord=2)
        denominator = np.linalg.norm(x,ord=2)
        normalized_error += numerator/denominator
    av_normalized_error = normalized_error/2000
    return av_normalized_error



3. Noiseless case: (n = 0)
Implement OMP (you may stop the OMP iterations once ∥y−Ax(k)∥2 is close to 0, here
x(k) is the estimation of x after kth iteration) and evaluate its performance. Calculate the
probability of Exact Support Recovery (i.e. the fraction of runs whenˆ
S= S) by averaging
over 2000 random realizations of A, as a function of M and smax (for different fixed values of
N). For each N, the probability of exact support recovery is a two dimensional plot (function
of M and smax) and you can display it as an image. The resulting plot is called the “noiseless
phase transition" plot, and it shows how many measurements (M) are needed for OMP to
successfully recover the sparse signal, as a function of smax. Do you observe a sharp transition
region where the probability quickly transitions from a large value (close to 1) to a very small
value (close to 0)? Generate different phase transition plots for the following values of N:
20, 50 and 100. Regenerate phase transition plots for average Normalized Error (instead of
probability of successful support recovery). Comment on both kinds of plots.

In [ ]:

noise = False


4. Noisy case: (n != 0)

In [ ]:
noise = True